## Goal:
Try to log all the function or submodules which are used while running a circuit, to detect faulty modules.

In [5]:
import cProfile
import re

import pstats
from pstats import SortKey

In [64]:
# inpit qasm

qasm_cirq = """
OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
creg meas[3];
h q;
measure q -> meas;
"""

qasm_ibm = """
OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
measure q -> c;
"""

In [1]:
from qiskit import QuantumCircuit, Aer, execute

# From str.
qc = QuantumCircuit.from_qasm_str(qasm_ibm)
# If you want to read from file, use instead
# qc = QuantumCircuit.from_qasm_file("/path/to/file.qasm")

# You can choose other backend also.
backend = Aer.get_backend("qasm_simulator") # aer_simulator


# Execute the circuit and show the result.
#job = execute(qc, backend)
#result = job.result()
#print(result.get_counts())

cProfile.run('my_job = execute(qc, backend)', filename='restats_qiskit')
result = my_job.result()
print(result.get_counts())

NameError: name 'qasm_ibm' is not defined

In [13]:
p = pstats.Stats('restats_qiskit')
# strip_dirs shortens the file path
p.strip_dirs().sort_stats(-1).print_stats()

Fri Oct 22 13:18:54 2021    restats_qiskit

         9601 function calls (9553 primitive calls) in 0.014 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(det)
        1    0.000    0.000    0.014    0.014 <string>:1(<module>)
        6    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
       33    0.000    0.000    0.000    0.000 __init__.py:1424(info)
        6    0.000    0.000    0.000    0.000 __init__.py:1663(getEffectiveLevel)
       39    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        6    0.000    0.000    0.000    0.000 __init__.py:214(_acquireLock)
        6    0.000    0.000    0.000    0.000 __init__.py:223(_releaseLock)
        1    0.000    0.000    0.000    0.000 _base.py:316(__init__)
        1    0.000    0.000    0.000    0.000 _collections_abc.py:657(get)
        1    0.000    0.000    0.000   

        5    0.000    0.000    0.000    0.000 typing.py:172(_collect_type_vars)
        2    0.000    0.000    0.000    0.000 typing.py:183(<listcomp>)
        4    0.000    0.000    0.000    0.000 typing.py:187(_subs_tvars)
        4    0.000    0.000    0.000    0.000 typing.py:206(_check_generic)
        5    0.000    0.000    0.000    0.000 typing.py:255(inner)
        4    0.000    0.000    0.000    0.000 typing.py:324(__eq__)
       35    0.000    0.000    0.000    0.000 typing.py:643(_is_dunder)
        5    0.000    0.000    0.000    0.000 typing.py:656(__init__)
       12    0.000    0.000    0.000    0.000 typing.py:666(<genexpr>)
        4    0.000    0.000    0.000    0.000 typing.py:674(__getitem__)
        9    0.000    0.000    0.000    0.000 typing.py:682(<genexpr>)
        5    0.000    0.000    0.000    0.000 typing.py:686(copy_with)
        4    0.000    0.000    0.000    0.000 typing.py:708(__eq__)
      8/7    0.000    0.000    0.000    0.000 typing.py:717(__hash__

In [84]:
qc.draw()

┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤M├───
     └───┘┌─┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─╫─┤M├
          └───┘ ║ └╥┘
c: 2/═══════════╩══╩═
                0  1

In [67]:
from cirq.contrib.qasm_import import circuit_from_qasm
circuit = circuit_from_qasm(qasm_ibm)
print(circuit)

q_0: ───H───@───M('c_0')───
            │
q_1: ───────X───M('c_1')───


In [69]:
import cirq
from cirq.ops.measurement_gate import MeasurementGate

def get_all_measurement_keys(circuit):
    all_ops = list(circuit.findall_operations_with_gate_type(MeasurementGate))
    return sorted([e[2].key for e in all_ops])

all_keys = get_all_measurement_keys(circuit)

In [83]:
from cirq import Simulator
simulator = Simulator()
samples = simulator.run(circuit, repetitions=1024)
# help(samples)
counter = samples.multi_measurement_histogram(keys=all_keys)
result = {"".join([str(d) for d in e]): int(v) for (e, v) in counter.items()}
result

{'11': 511, '00': 513}

In [11]:
p.print_callees()

   Ordered by: standard name

Function                                                                     called...
                                                                                 ncalls  tottime  cumtime
<__array_function__ internals>:2(det)                                        ->       2    0.000    0.000  linalg.py:472(_unary_dispatcher)
                                                                                      2    0.000    0.000  {built-in method numpy.core._multiarray_umath.implement_array_function}
<string>:1(<module>)                                                         ->       1    0.000    0.014  execute_function.py:39(execute)
__init__.py:1412(debug)                                                      ->       6    0.000    0.000  __init__.py:1677(isEnabledFor)
__init__.py:1424(info)                                                       ->      33    0.000    0.000  __init__.py:1677(isEnabledFor)
__init__.py:1677(isEnabledFor)             